In [ ]:
# Install dependencies (run this only once)
# !pip install aif360 scikit-learn seaborn matplotlib

import pandas as pd
import numpy as np
from aif360.datasets import CompasDataset
from aif360.metrics import BinaryLabelDatasetMetric, ClassificationMetric
from aif360.algorithms.preprocessing import Reweighing
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns

# Load COMPAS dataset
dataset = CompasDataset()

# Split into train and test sets
train, test = dataset.split([0.7], shuffle=True)

# Bias metrics before mitigation
metric_orig = BinaryLabelDatasetMetric(train, unprivileged_groups=[{'race': 1}], privileged_groups=[{'race': 0}])
print("Original disparate impact:", metric_orig.disparate_impact())

# Apply reweighing
RW = Reweighing(unprivileged_groups=[{'race': 1}], privileged_groups=[{'race': 0}])
RW.fit(train)
train_transf = RW.transform(train)

# Prepare data for classifier
X_train = train_transf.features
y_train = train_transf.labels.ravel()
X_test = test.features
y_test = test.labels.ravel()

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Train model
model = LogisticRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

# Predict on test set
test_pred = test.copy()
test_pred.labels = y_pred

# Evaluate fairness post-mitigation
metric_post = ClassificationMetric(test, test_pred,
                                   unprivileged_groups=[{'race': 1}],
                                   privileged_groups=[{'race': 0}])

print("Disparate Impact (after):", metric_post.disparate_impact())
print("Statistical Parity Difference (after):", metric_post.statistical_parity_difference())
print("Accuracy:", accuracy_score(y_test, y_pred))

# Plot false positive rate by race
races = test_pred.protected_attributes[:, 0]
df = pd.DataFrame({
    'race': races,
    'true': test.labels.ravel(),
    'pred': y_pred
})
df['false_positive'] = (df['pred'] == 1) & (df['true'] == 0)
df['race_label'] = df['race'].map({0.0: 'Caucasian', 1.0: 'African-American'})

sns.barplot(data=df, x='race_label', y='false_positive', estimator=np.mean)
plt.title('False Positive Rate by Race')
plt.ylabel('False Positive Rate')
plt.xlabel('Race')
plt.show()
